# Блок №4

# Задание 1
___
Сопоставьте типы данных с подходящими значениями:

Ответ:
+ DateTime64 -- 2019-12-31 03:00:00.000
+ date -- 2019-12-31
+ Int8 -- -120
+ UInt8 -- 220
+ UInt64 -- 17441114072349551615

# Задание 2
___
Какую из представленных функций часто используют для приведения типов?

Ответ:
+ CAST

# Задание 3
___
Далее в заданиях используются данные с платформы [Airbnb](http://insideairbnb.com/index.html) в Берлине ✈️.

Имеются следующие таблицы:

1) `listings` – информация о жилье, включая полные описания, характеристики и средние оценки в отзывах; поскольку столбцов очень много, нужные перечислены в текстах самих задач

2) `calendar_summary` – информация о доступности и цене того или иного жилья по дням

+ `listing_id` – идентификатор жилья (объявления)
+ `date` – дата
+ `available` – доступность жилья в данный день (t/f)
+ `price` – цена за ночь

3)`reviews` – отзывы

+ `listing_id` –  идентификатор жилья  
+ `id` – id отзыва
+ `date` – дата отзыва
+ `reviewer_id` – id ревьюера (автора отзыва)
+ `reviewer_name` – имя автора
+ `comments` – сам отзыв

На Airbnb есть две основные группы пользователей:

+ Хозяева (хосты, hosts) – сдают жилье
+ Путешественники (travelers) – снимают

Начнем с анализа характеристик хозяев в таблице `listings`! 

Пользователи, сдающие квартиры на Airbnb, зарегистрировались в разное время. Кто-то – очень давно, а кто-то совсем недавно. Давайте проверим, в какой месяц и год зарегистрировалось наибольшее количество новых хостов. В качестве ответа введите дату следующего формата: `2010-12`

+ `host_id` – идентификатор хозяина
+ `host_since` – дата регистрации как хост

Сам хост может встретиться в таблице несколько раз, если сдает несколько помещений, поэтому не забудьте оставить уникальные значения `host_id`. Также обратите внимание на тип данных в `host_since`, возможно вам пригодится `toStartOfMonth()` для извлечения части даты и `toDateOrNull()` для обработки пустых значений.

Правильный запрос:
```
SELECT 
    toStartOfMonth(toDateOrNull(host_since)) as StartMonth,
    COUNT(DISTINCT host_id) AS NumHosts
FROM
    listings 
GROUP BY
    StartMonth
ORDER BY
    NumHosts DESC
LIMIT 10
```
Ответ:
```
2015-07
```

# Задание 4
___
Посмотрим на среднюю частоту ответа среди хозяев (f) и суперхозяев (t).

Значения частоты ответа хранятся как строки и включают значок %, который необходимо заменить на пустоту (''). После этого приведите столбец к нужному типу данных с помощью `toInt32OrNull()` и посчитайте среднюю частоту отклика в разбивке по тому, является ли хост суперхозяином или нет. В качестве ответа укажите наибольшее среднее.

+ `host_response_rate` – частота ответа
+ `host_is_superhost` – является ли суперхозяином

Важный момент: у каждого `host_id` есть только одно уникальное значение частоты ответа, а также одна единая отметка суперхозяина. Чтобы посчитать показатели честно, нужно использовать подзапрос и взять уникальные комбинации с помощью `DISTINCT`. Примерная структура финального запроса:
```
SELECT
    ... -- перечисляем все нужные колонки и операции
FROM (
    SELECT 
        DISTINCT column_1,
        column_2,
        column_3
    FROM 
        table_name) AS sub -- в подзапросе выбираем уникальные значения строк по интересующим колонкам
... -- не забываем LIMIT, сортировку и вот это вот всё
```
Пригодятся:

+ replaceAll(столбец, что_заменить, на_что_заменить) – замена значений в колонке
+ toIntOrNull(столбец)

Суперхозяева — опытные хозяева с отличными отзывами. В задаче используется таблица listings.

Правильный запрос:
```
SELECT
    host_is_superhost,
    AVG(toInt32OrNull(replaceAll(host_response_rate, '%', ''))) as AvgResp,
    MIN(toInt32OrNull(replaceAll(host_response_rate, '%', ''))) as MinResp,
    MAX(toInt32OrNull(replaceAll(host_response_rate, '%', ''))) as MaxResp
FROM
    (SELECT 
        DISTINCT 
            host_id,
            host_response_rate,
            host_is_superhost
    FROM listings 
    ) as sub

WHERE 
    host_is_superhost != ''
GROUP BY
    host_is_superhost
LIMIT 10
```
Ответ:
```
98
```

# Задание 5
___
Сгруппируйте данные из listings по хозяевам (`host_id`) и посчитайте, какую цену за ночь в среднем каждый из них устанавливает (у одного хоста может быть несколько объявлений). Идентификаторы сдаваемого жилья объедините в отдельный массив. Таблицу отсортируйте по убыванию средней цены и убыванию `host_id` (в таком порядке). В качестве ответа укажите первый массив в результирующей таблице, состоящий более чем из двух id. Например:

> 2343477,6340,32324

+ `host_id` – идентификатор хозяина
+ `id` – идентификатор жилья
+ `price` – цена за ночь в конкретном месте

Пригодятся:

+ groupArray(столбец) – составление массива
+ replaceRegexpAll(столбец, что_заменить, на_что_заменить) – паттерн `[$,]` с использованием функции `replaceRegexpAll()` чтобы избавиться от $ и запятых

Пример того, как выглядят цены в колонке price:

| Price    | 
|:--------:|
| `$`16.00   | 
| `$`2,228.00|  


+ toFloat32OrNull(столбец)

id нужно вставить без пробелов, через запятую. 

Правильный запрос:
```
SELECT 
    host_id,
    groupArray(id) as host_listings,
    groupArray(price) as prices,
    AVG(toFloat64OrNull(replaceRegexpAll(price, '[$,]', ''))) as AvgPricePerHost
FROM
    listings
GROUP BY
    host_id
ORDER BY
    AvgPricePerHost DESC
LIMIT 10
```
Ответ:
```
25757977,25759146,25802565,25802651,25802838,25802909,25803050,25803117,25803218,25803260
```

# Задание 6
___
Немного усложним предыдущую задачу, и посчитаем разницу между максимальной и минимальной установленной ценой у каждого хозяина. В качестве ответа укажите идентификатор хоста, у которого разница оказалась наибольшей. 

+ `host_id` – идентификатор хозяина
+ `id` – идентификатор жилья
+ `price` – цена за ночь в конкретном месте

Пригодятся:

+ replaceRegexpAll(столбец, что_заменить, на_что_заменить) – паттерн `[$,]` чтобы избавиться от $ и запятых
+ toFloat32OrNull(столбец)

Note: не забудьте повторить операции с предыдущего шага, обратите внимание на сортировку.

Правильный запрос:
```
SELECT 
    host_id,
    groupArray(id) as host_listings,
    MIN(toFloat64OrNull(replaceRegexpAll(price, '[$,]', ''))) as MinPricePerHost,
    MAX(toFloat64OrNull(replaceRegexpAll(price, '[$,]', ''))) as MaxPricePerHost,
    MaxPricePerHost - MinPricePerHost AS price_diff
FROM
    listings
GROUP BY
    host_id
ORDER BY
    price_diff DESC
LIMIT 10
```
Ответ:
```
155140624
```

# Задание 7
___
Теперь сгруппируйте данные по типу жилья и выведите средние значения цены за ночь, размера депозита и цены уборки. Обратите внимание на тип данных, наличие значка $ и запятых в больших суммах. Для какого типа жилья среднее значение залога наибольшее?

+ `room_type` – тип сдаваемого жилья 
+ `price` – цена за ночь
+ `security_deposit` – залог за сохранность имущества
+ `cleaning_fee` – плата за уборку

Правильный запрос:
```
SELECT
    room_type,
    AVG(toFloat32OrNull(replaceRegexpAll(price, '[$,]', ''))) as AvgPrice,
    AVG(toFloat32OrNull(replaceRegexpAll(security_deposit, '[$,]', ''))) as AvgSecurityDeposit,
    AVG(toFloat32OrNull(replaceRegexpAll(cleaning_fee, '[$,]', ''))) as AvgClean
FROM
    listings
GROUP BY
    room_type
ORDER BY
    AvgSecurityDeposit DESC
```
Ответ:
```
Entire home/apt
```

# Задание 8
___
В каких частях города средняя стоимость за ночь является наиболее низкой? 

Сгруппируйте данные по `neighbourhood_cleansed` и посчитайте среднюю цену за ночь в каждом районе. В качестве ответа введите название места, где средняя стоимость за ночь ниже всего.

+ `price` – цена за ночь
+ `neighbourhood_cleansed` – район/округ города

Правильный запрос:
```
SELECT
    neighbourhood_cleansed,
    AVG(toFloat32OrNull(replaceRegexpAll(price, '[$,]', ''))) as AvgPrice,
    MAX(toFloat32OrNull(replaceRegexpAll(price, '[$,]', ''))) as MaxPrice,
    MIN(toFloat32OrNull(replaceRegexpAll(price, '[$,]', ''))) as MinPrice
FROM
    listings
GROUP BY
    neighbourhood_cleansed 
ORDER BY
    AvgPrice
LIMIT 10
```
Ответ:
```
Neu-Hohenschönhausen Süd
```

# Задание 9
___
В каких районах Берлина средняя площадь жилья, которое сдаётся целиком, является наибольшей? Отсортируйте по среднему и выберите топ-3. 

+ `neighbourhood_cleansed` – район
+ `square_feet` – площадь
+ `room_type` – тип (нужный – 'Entire home/apt')

Правильный запрос:
```
SELECT
    neighbourhood_cleansed,
    AVG(toFloat32OrNull(square_feet)) as AvgSquare
FROM
    listings
WHERE
    room_type = 'Entire home/apt'
GROUP BY
    neighbourhood_cleansed 
ORDER BY
    AvgSquare DESC
LIMIT 10
```
Ответ:

+ Kaulsdorf
+ Schöneberg-Süd 
+ Lichtenrade


# Задание 10
___
Напоследок давайте посмотрим, какая из представленных комнат расположена ближе всего к центру города! В качестве ответа укажите id объявления.

+ `id` – идентификатор жилья (объявления)
+ `room_type` – тип жилья ('Private room')
+ `latitude` – широта
+ `longitude` – долгота
+ 52.5200 с.ш., 13.4050 в.д – координаты центра Берлина

Правильный запрос:
```
SELECT 
    id,
    host_id,
    toFloat64OrNull(latitude) as latitude,
    toFloat64OrNull(longitude) as longitude,
    geoDistance(13.4050, 52.5200, longitude, latitude) as dist
FROM
    listings
WHERE 
    room_type = 'Private room'
ORDER BY
    dist 
LIMIT 1
```
Ответ:
```
19765058
```